# Ackley Function Optimization

## 1. CUDA Parallel Implementation

In [12]:

%%writefile ackley_cuda.cu
#include <iostream>
#include <cmath>
#include <curand_kernel.h>
#include <cfloat>
#include <chrono>
#include <fstream>  // Required for CSV file I/O
#include <iomanip>  // Required for precision

using namespace std;

// ------------------------------
// PARAMETERS
// ------------------------------
#define POP 256
#define DIM 5
#define MAX_IT 8000
#define LB -100.0
#define UB 100.0

#define BLOCK_SIZE 256

// ------------------------------
// DEVICE HELPER FUNCTIONS
// ------------------------------
// Device function for generating random doubles
__device__ double randF(curandState* state, double a, double b) {
    double r = curand_uniform_double(state);
    return a + r * (b - a);
}

// Device function for generating random integers
__device__ int randInt(curandState* state, int a, int b) {
    float r = curand_uniform(state);
    return (int)(a + r * (b - a + 0.99999f));
}

// ------------------------------
// FITNESS FUNCTION: ACKLEY
// ------------------------------
__device__ double fitness(double* x, int dim) {
    double sumsq = 0.0, sumcos = 0.0;
    const double PI = 3.14159265358979323846;
    for (int i = 0; i < dim; i++) {
        sumsq += x[i] * x[i];
        sumcos += cos(2.0 * PI * x[i]);
    }
    double term1 = -20.0 * exp(-0.2 * sqrt(sumsq / dim));
    double term2 = -exp(sumcos / dim);
    return term1 + term2 + 20.0 + exp(1.0);
}

// ------------------------------
// KERNELS
// ------------------------------
__global__ void init_population_kernel(double* pop, curandState* states, unsigned long seed) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < POP) {
        curand_init(seed, idx, 0, &states[idx]);
        for (int d = 0; d < DIM; d++) {
            pop[idx * DIM + d] = randF(&states[idx], LB, UB);
        }
    }
}

__global__ void evaluate_fitness_kernel(double* pop, double* fitness_vals) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < POP) {
        double sol[DIM];
        for (int d = 0; d < DIM; d++) {
            sol[d] = pop[idx * DIM + d];
        }
        fitness_vals[idx] = fitness(sol, DIM);
    }
}

__global__ void loa_update_kernel(double* pop, double* fitness_vals, double* new_pop, double* new_fitness_vals, curandState* states, int t) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < POP) {
        curandState localState = states[idx];

        // 1. Find better candidates
        int betterIdx = -1;
        int betterCount = 0;

        for (int j = 0; j < POP; j++) {
            if (fitness_vals[j] < fitness_vals[idx]) {
                betterCount++;
            }
        }

        if (betterCount > 0) {
            int pick = randInt(&localState, 0, betterCount - 1);
            int current = 0;
            for (int j = 0; j < POP; j++) {
                if (fitness_vals[j] < fitness_vals[idx]) {
                    if (current == pick) {
                        betterIdx = j;
                        break;
                    }
                    current++;
                }
            }
        }

        // 2. Generate Candidate
        double candidate[DIM];
        double currentPos[DIM];
        for(int d=0; d<DIM; d++) currentPos[d] = pop[idx * DIM + d];

        double rp = curand_uniform_double(&localState);

        if (rp < 0.5 && betterIdx != -1) {
            // Escape Move
            double betterPos[DIM];
            for(int d=0; d<DIM; d++) betterPos[d] = pop[betterIdx * DIM + d];

            for (int d = 0; d < DIM; d++) {
                double r = curand_uniform_double(&localState);
                int I = randInt(&localState, 1, 2);
                candidate[d] = currentPos[d] + r * (betterPos[d] - I * currentPos[d]);

                // Bounds
                if (candidate[d] < LB) candidate[d] = LB;
                if (candidate[d] > UB) candidate[d] = UB;
            }
        } else {
            // Hide Move
            for (int d = 0; d < DIM; d++) {
                double r = curand_uniform_double(&localState);
                candidate[d] = currentPos[d] + (1.0 - 2.0 * r) * (UB - LB) / (double)t;

                // Bounds
                if (candidate[d] < LB) candidate[d] = LB;
                if (candidate[d] > UB) candidate[d] = UB;
            }
        }

        // 3. Evaluate Candidate
        double f = fitness(candidate, DIM);

        // 4. Update if better
        if (f < fitness_vals[idx]) {
            for(int d=0; d<DIM; d++) new_pop[idx * DIM + d] = candidate[d];
            new_fitness_vals[idx] = f;
        } else {
            for(int d=0; d<DIM; d++) new_pop[idx * DIM + d] = currentPos[d];
            new_fitness_vals[idx] = fitness_vals[idx];
        }

        states[idx] = localState; // Save state back
    }
}

// ------------------------------
// MAIN FUNCTION
// ------------------------------
int main() {
    auto t_start = chrono::high_resolution_clock::now();

    // 1. Initialize CSV File
    ofstream csvFile("ackley_data.csv");
    // Write Header
    csvFile << "Iteration,Best_Fitness";
    for(int d=0; d<DIM; d++) csvFile << ",x" << d+1;
    csvFile << "\n";
    // Set high precision for scientific data
    csvFile << fixed << setprecision(10);

    double *d_pop, *d_fitness, *d_new_pop, *d_new_fitness;
    curandState *d_states;

    // Host buffers for logging
    double h_fitness[POP];
    double h_pop[POP * DIM];

    size_t pop_size = POP * DIM * sizeof(double);
    size_t fit_size = POP * sizeof(double);

    cudaMalloc(&d_pop, pop_size);
    cudaMalloc(&d_fitness, fit_size);
    cudaMalloc(&d_new_pop, pop_size);
    cudaMalloc(&d_new_fitness, fit_size);
    cudaMalloc(&d_states, POP * sizeof(curandState));

    int threads = 128;
    int blocks = (POP + threads - 1) / threads;

    init_population_kernel<<<blocks, threads>>>(d_pop, d_states, time(NULL));
    evaluate_fitness_kernel<<<blocks, threads>>>(d_pop, d_fitness);
    cudaDeviceSynchronize();

    double bestGlobalFit = DBL_MAX;

    cout << "Starting optimization for " << MAX_IT << " iterations..." << endl;

    // ------------------------------
    // OPTIMIZATION LOOP
    // ------------------------------
    for (int it = 1; it <= MAX_IT; it++) {
        // A. Run Optimization Step on GPU
        loa_update_kernel<<<blocks, threads>>>(d_pop, d_fitness, d_new_pop, d_new_fitness, d_states, it);
        cudaDeviceSynchronize();

        // B. Swap Pointers
        double *temp_pop = d_pop; d_pop = d_new_pop; d_new_pop = temp_pop;
        double *temp_fit = d_fitness; d_fitness = d_new_fitness; d_new_fitness = temp_fit;

        // C. Data Logging (HAPPENS EVERY ITERATION)
        // Copy current generation data to CPU
        cudaMemcpy(h_fitness, d_fitness, fit_size, cudaMemcpyDeviceToHost);
        cudaMemcpy(h_pop, d_pop, pop_size, cudaMemcpyDeviceToHost);

        // Find the best individual in this specific generation
        int bestIdx = 0;
        for(int i=1; i<POP; i++) {
            if(h_fitness[i] < h_fitness[bestIdx]) {
                bestIdx = i;
            }
        }

        // Track global best for console output
        if(h_fitness[bestIdx] < bestGlobalFit) bestGlobalFit = h_fitness[bestIdx];

        // Write to CSV: Iteration, Fitness, x1, x2, x3, x4, x5
        csvFile << it << "," << h_fitness[bestIdx];
        for(int d=0; d<DIM; d++) {
            csvFile << "," << h_pop[bestIdx * DIM + d];
        }
        csvFile << "\n";

        // D. Console Progress
        if (it % 1000 == 0) {
             cout << "Iter " << it << " | Best = " << bestGlobalFit << endl;
        }
    }

    csvFile.close();

    // ------------------------------
    // FINAL OUTPUT
    // ------------------------------
    cout << "\nOptimization Complete." << endl;
    cout << "Data saved to 'ackley_data.csv' (8000 rows)" << endl;
    cout << "Final Best Value = " << bestGlobalFit << endl;

    // Cleanup
    cudaFree(d_pop);
    cudaFree(d_fitness);
    cudaFree(d_new_pop);
    cudaFree(d_new_fitness);
    cudaFree(d_states);

    auto t_end = chrono::high_resolution_clock::now();
    double elapsed = chrono::duration_cast<chrono::duration<double>>(t_end - t_start).count();
    cout << "Execution Time = " << elapsed << " sec" << endl;

    return 0;
}

Overwriting ackley_cuda.cu


In [13]:
!nvcc -arch=sm_75 ackley_cuda.cu -o ackley_cuda
!./ackley_cuda

Starting optimization for 8000 iterations...
Iter 1000 | Best = 4.44089e-16
Iter 2000 | Best = 4.44089e-16
Iter 3000 | Best = 4.44089e-16
Iter 4000 | Best = 4.44089e-16
Iter 5000 | Best = 4.44089e-16
Iter 6000 | Best = 4.44089e-16
Iter 7000 | Best = 4.44089e-16
Iter 8000 | Best = 4.44089e-16

Optimization Complete.
Data saved to 'ackley_data.csv' (8000 rows)
Final Best Value = 4.44089e-16
Execution Time = 0.851905 sec


## 2. Serial (Sequential) Implementation

In [14]:

%%writefile ackley_serial.cpp
#include <bits/stdc++.h>
using namespace std;

/* ---------------------------------------
      Random Generator
---------------------------------------*/
mt19937 rng(time(NULL));

double randF(double a, double b) {
    uniform_real_distribution<double> dist(a, b);
    return dist(rng);
}
int randInt(int a, int b) {
    uniform_int_distribution<int> dist(a, b);
    return dist(rng);
}

/* ---------------------------------------
            Fitness Function
---------------------------------------*/
double ackley(const vector<double> &x) {
    int dim = x.size();

    double sumsq = 0.0, sumcos = 0.0;
    const double PI = 3.14159265358979323846;
    for (int i = 0; i < dim; i++) {
        sumsq += x[i] * x[i];
        sumcos += cos(2.0 * PI * x[i]);
    }
    double term1 = -20.0 * exp(-0.2 * sqrt(sumsq / dim));
    double term2 = -exp(sumcos / dim);
    return term1 + term2 + 20.0 + exp(1.0);

}

/* ---------------------------------------
            LOA PARAMETERS
---------------------------------------*/
int POP = 256;
int DIM = 5;
int MAX_IT = 8000;
double LB = -100;
double UB = 100;

/* ---------------------------------------
     Escape (Global Search)
---------------------------------------*/
vector<double> escape(const vector<double> &x,
                      const vector<double> &SSA)
{
    vector<double> newX = x;
    for(int j=0;j<DIM;j++){
        double r = randF(LB, UB); // Note: Original code used randF(rng,0,1) here but logic was r*(SSA-I*x).
        // Wait, original code: double r = randF(rng,0,1);
        // Let's stick to original logic.
        double r_val = randF(0, 1);
        int I = randInt(1, 2);
        newX[j] = x[j] + r_val * (SSA[j] - I*x[j]);

        newX[j] = min(max(newX[j],LB),UB);
    }
    return newX;
}

/* ---------------------------------------
     Hide (Local Search)
---------------------------------------*/
vector<double> hide(const vector<double> &Xi,int t){
    vector<double> newX = Xi;

    for(int j=0;j<DIM;j++){
        double r = randF(0, 1);
        newX[j] = Xi[j] + (1 - 2*r)*(UB-LB)/t;
        newX[j] = min(max(newX[j],LB),UB);
    }
    return newX;
}

/* ---------------------------------------
         Initialize population
---------------------------------------*/
vector<vector<double>> init_population(){
    vector<vector<double>> pop(POP, vector<double>(DIM));
    for(int i=0;i<POP;i++)
        for(int d=0;d<DIM;d++)
            pop[i][d] = randF(LB, UB);
    return pop;
}

/* =======================================
             MAIN LOA SERIAL
=======================================*/
int main(){
    auto t1 = chrono::high_resolution_clock::now();

    vector<vector<double>> pop = init_population();
    vector<double> fitness(POP);

    // Initial fitness
    for(int i=0;i<POP;i++) fitness[i]=ackley(pop[i]);

    double bestFit = 1e18;
    vector<double> bestSol(DIM);

    // find initial best
    for(int i=0;i<POP;i++){
        if(fitness[i]<bestFit){
            bestFit=fitness[i];
            bestSol=pop[i];
        }
    }

    /* --------------------------------------
                LOA ITERATIONS
    ---------------------------------------*/
    for(int it=1; it<=MAX_IT; it++){

        for(int i=0;i<POP;i++){

            vector<int> better;
            for(int j=0;j<POP;j++)
                if(fitness[j]<fitness[i]) better.push_back(j);

            int betterIdx=-1;
            if(!better.empty())
                betterIdx = better[randInt(0,(int)better.size()-1)];

            vector<double> candidate;
            if(randF(0,1)<0.5 && betterIdx!=-1)
                candidate = escape(pop[i],pop[betterIdx]);
            else
                candidate = hide(pop[i],it);

            double f = ackley(candidate);

            if(f<fitness[i]){
                pop[i]=candidate;
                fitness[i]=f;
            }
            if(f<bestFit){
                bestFit=f;
                bestSol=candidate;
            }
        }

        if(it % 1000 == 0)
            cout<<"Iter "<<it<<" | Best = "<<bestFit<<"\n";
    }

    cout<<"\nFinal Best Solution:\n";
    for(int i=0;i<DIM;i++) cout<<"x"<<i+1<<" = "<<bestSol[i]<<endl;
    cout<<"\nBest " << "Ackley" << " Value = "<<bestFit<<endl;

    auto t2 = chrono::high_resolution_clock::now();
    cout<<"\nExecution Time = "
        <<chrono::duration<double>(t2-t1).count()
        <<" sec\n";

    return 0;
}


Writing ackley_serial.cpp


In [15]:
!g++ ackley_serial.cpp -o ackley_serial
!./ackley_serial

Iter 1000 | Best = 4.44089e-16
Iter 2000 | Best = 4.44089e-16
Iter 3000 | Best = 4.44089e-16
Iter 4000 | Best = 4.44089e-16
Iter 5000 | Best = 4.44089e-16
Iter 6000 | Best = 4.44089e-16
Iter 7000 | Best = 4.44089e-16
Iter 8000 | Best = 4.44089e-16

Final Best Solution:
x1 = 1.99752e-17
x2 = 7.08364e-17
x3 = -4.56126e-17
x4 = 1.47864e-16
x5 = 5.78265e-16

Best Ackley Value = 4.44089e-16

Execution Time = 6.83021 sec


## 3. OpenMP Parallel Implementation

In [16]:

%%writefile ackley_omp.cpp
#include <bits/stdc++.h>
#include <omp.h>
using namespace std;

/* ---------------------------------------
      Thread-safe Random Generator
---------------------------------------*/
double randF(mt19937 &rng, double a, double b) {
    uniform_real_distribution<double> dist(a, b);
    return dist(rng);
}
int randInt(mt19937 &rng, int a, int b) {
    uniform_int_distribution<int> dist(a, b);
    return dist(rng);
}

/* ---------------------------------------
            Fitness Function
---------------------------------------*/
double ackley(const vector<double> &x) {
    int dim = x.size();

    double sumsq = 0.0, sumcos = 0.0;
    const double PI = 3.14159265358979323846;
    for (int i = 0; i < dim; i++) {
        sumsq += x[i] * x[i];
        sumcos += cos(2.0 * PI * x[i]);
    }
    double term1 = -20.0 * exp(-0.2 * sqrt(sumsq / dim));
    double term2 = -exp(sumcos / dim);
    return term1 + term2 + 20.0 + exp(1.0);

}

/* ---------------------------------------
            LOA PARAMETERS
---------------------------------------*/
int POP = 256;
int DIM = 5;
int MAX_IT = 8000;
double LB = -100;
double UB = 100;

/* ---------------------------------------
     Escape (Global Search)  — parallel safe
---------------------------------------*/
vector<double> escape(const vector<double> &x,
                      const vector<double> &SSA,
                      mt19937 &rng)
{
    vector<double> newX = x;
    for(int j=0;j<DIM;j++){
        double r = randF(rng,0,1);
        int I = randInt(rng,1,2);
        newX[j] = x[j] + r * (SSA[j] - I*x[j]);

        newX[j] = min(max(newX[j],LB),UB);
    }
    return newX;
}

/* ---------------------------------------
     Hide (Local Search) — parallel safe
---------------------------------------*/
vector<double> hide(const vector<double> &Xi,int t,mt19937 &rng){
    vector<double> newX = Xi;

    for(int j=0;j<DIM;j++){
        double r = randF(rng,0,1);
        newX[j] = Xi[j] + (1 - 2*r)*(UB-LB)/t;
        newX[j] = min(max(newX[j],LB),UB);
    }
    return newX;
}

/* ---------------------------------------
         Initialize population (PARALLEL)
---------------------------------------*/
vector<vector<double>> init_population(vector<mt19937> &rngs){
    vector<vector<double>> pop(POP, vector<double>(DIM));

    #pragma omp parallel
    {
        int tid = omp_get_thread_num();
        mt19937 &local_rng = rngs[tid];

        #pragma omp for schedule(static)
        for(int i=0;i<POP;i++)
            for(int d=0;d<DIM;d++)
                pop[i][d] = randF(local_rng,LB,UB);
    }
    return pop;
}

/* =======================================
             MAIN LOA PARALLEL
=======================================*/
int main(){
    auto t1 = chrono::high_resolution_clock::now();

    int threads = omp_get_max_threads();
    vector<mt19937> rngs(threads);

    random_device rd;
    for(int i=0;i<threads;i++)
        rngs[i].seed(rd()+i*111);

    vector<vector<double>> pop = init_population(rngs);
    vector<double> fitness(POP);

    // Initial fitness
    for(int i=0;i<POP;i++) fitness[i]=ackley(pop[i]);

    double bestFit = 1e18;
    vector<double> bestSol(DIM);

    // find initial best
    for(int i=0;i<POP;i++){
        if(fitness[i]<bestFit){
            bestFit=fitness[i];
            bestSol=pop[i];
        }
    }

    /* --------------------------------------
                LOA ITERATIONS
       Full population parallel every step
    ---------------------------------------*/
    for(int it=1; it<=MAX_IT; it++){

        #pragma omp parallel
        {
            int tid = omp_get_thread_num();
            mt19937 &localRng = rngs[tid];

            double localBest = 1e18;
            vector<double> localBestSol(DIM);

            #pragma omp for schedule(static)
            for(int i=0;i<POP;i++){

                vector<int> better;
                for(int j=0;j<POP;j++)
                    if(fitness[j]<fitness[i]) better.push_back(j);

                int betterIdx=-1;
                if(!better.empty())
                    betterIdx = better[randInt(localRng,0,(int)better.size()-1)];

                vector<double> candidate;
                if(randF(localRng,0,1)<0.5 && betterIdx!=-1)
                    candidate = escape(pop[i],pop[betterIdx],localRng);
                else
                    candidate = hide(pop[i],it,localRng);

                double f = ackley(candidate);

                if(f<fitness[i]){
                    pop[i]=candidate;
                    fitness[i]=f;
                }
                if(f<localBest){
                    localBest=f;
                    localBestSol=candidate;
                }
            }

            // Update global best safely
            #pragma omp critical
            {
                if(localBest<bestFit){
                    bestFit=localBest;
                    bestSol=localBestSol;
                }
            }
        }

        if(it % 1000 == 0)
            cout<<"Iter "<<it<<" | Best = "<<bestFit<<"\n";
    }

    cout<<"\nFinal Best Solution:\n";
    for(int i=0;i<DIM;i++) cout<<"x"<<i+1<<" = "<<bestSol[i]<<endl;
    cout<<"\nBest " << "Ackley" << " Value = "<<bestFit<<endl;

    auto t2 = chrono::high_resolution_clock::now();
    cout<<"\nExecution Time = "
        <<chrono::duration<double>(t2-t1).count()
        <<" sec\n";

    return 0;
}


Writing ackley_omp.cpp


In [17]:
!g++ -fopenmp ackley_omp.cpp -o ackley_omp
!./ackley_omp

Iter 1000 | Best = 3.9968e-15
Iter 2000 | Best = 3.9968e-15
Iter 3000 | Best = 3.9968e-15
Iter 4000 | Best = 3.9968e-15
Iter 5000 | Best = 3.9968e-15
Iter 6000 | Best = 3.9968e-15
Iter 7000 | Best = 3.9968e-15
Iter 8000 | Best = 3.9968e-15

Final Best Solution:
x1 = 1.48387e-15
x2 = 4.56325e-16
x3 = 6.60433e-16
x4 = -9.13743e-16
x5 = 1.79591e-17

Best Ackley Value = 3.9968e-15

Execution Time = 5.40686 sec
